,content,raw_targets,domain,opinion_words,doc,predicted_targets
0,"this is an edited review , now that i have had...",[],MP3 player,[],<stanfordnlp.pipeline.doc.Document object at 0...,[]
1,"while , there are flaws with the machine , the...",[affordability],MP3 player,[flaws],<stanfordnlp.pipeline.doc.Document object at 0...,[affordability]
10,"for me , i 'll save that cash for another purc...",[],MP3 player,[],<stanfordnlp.pipeline.doc.Document object at 0...,[]
100,"the face-plate , which pops out for the batter...",[face plate],MP3 player,[],<stanfordnlp.pipeline.doc.Document object at 0...,[]
1000,after i reviewed the manual ( a pdf file inclu...,[],MP3 player,"[better, problems, warned]",<stanfordnlp.pipeline.doc.Document object at 0...,[]
1001,"so if you have xp , be prepared for this and d...",[],MP3 player,[alarmed],<stanfordnlp.pipeline.doc.Document object at 0...,[]
1002,if you are out for an mp3 player with a lot of...,[],MP3 player,"[great, recommend]",<stanfordnlp.pipeline.doc.Document object at 0...,[]
1003,i would have liked to have given it 4 1/2 star...,[size],MP3 player,[liked],<stanfordnlp.pipeline.doc.Document object at 0...,[]
1004,i got this a while ago knowing nothing about it .,[],MP3 player,[],<stanfordnlp.pipeline.doc.Document object at 0...,[]
1005,it is a beast .,[],MP3 player,[],<stanfordnlp.pipeline.doc.Document object at 0...,[]


In [50]:
import pickle

filepath = 'temp.pkl'
with open(filepath, 'wb') as f:
    pickle.dump(domain_pattern_counter, f)
    
with open(filepath, 'rb') as f:
    current_pkl = pickle.load(f)

In [2]:
import os
import pandas as pd
filepath = os.path.join(os.path.dirname(os.getcwd()), 'output', 'preliminary', 'sub', '[MP3 player]patterns_496.csv')
pattern_df = pd.read_csv(filepath)
domain_pattern_counter = {k:v for k, v in zip(pattern_df['pattern'], pattern_df['count'])}

In [35]:
from config import parameters
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8, progress_bar=True)

def match_opinion_words(content, opinion_word_lexicon):
    opinion_words = []
    for opinion in opinion_word_lexicon:
        for token in content.split():
            if token == opinion: opinion_words.append(token)
    return list(set(opinion_words))

df = pd.read_json(parameters.data_filepath)
opinion_word_lexicon = [item for sublist in pd.read_json(parameters.lexicon_filepath).values for item in sublist]
df['opinion_words'] = df.parallel_apply(lambda x: match_opinion_words(x['content'], opinion_word_lexicon), axis=1)
domain_df = df[df['domain']==df['domain'].unique()[0]]
domain_df.head()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,content,raw_targets,filename,domain,opinion_words
0,"this is an edited review , now that i have had...",[],Creative Labs Nomad Jukebox Zen Xtra 40GB,MP3 player,[]
1,"while , there are flaws with the machine , the...",[affordability],Creative Labs Nomad Jukebox Zen Xtra 40GB,MP3 player,[flaws]
10,"for me , i 'll save that cash for another purc...",[],Creative Labs Nomad Jukebox Zen Xtra 40GB,MP3 player,[]
100,"the face-plate , which pops out for the batter...",[face plate],Creative Labs Nomad Jukebox Zen Xtra 40GB,MP3 player,[]
1000,after i reviewed the manual ( a pdf file inclu...,[],Creative Labs Nomad Jukebox Zen Xtra 40GB,MP3 player,"[better, problems, warned]"


In [20]:
import stanfordnlp
nlp = stanfordnlp.Pipeline()

from DependencyGraph import DependencyGraph
def next_focused_tokens(current_tokens, token2idx, nodes, dep_rel):
    focused_tokens = set()
    for current_token in current_tokens:
        indices = token2idx[current_token]
        for current_token_idx in indices:

            if nodes[current_token_idx].dep == dep_rel: focused_tokens.add(nodes[nodes[current_token_idx].governor].token)
            child_nodes = [nodes[i] for i in range(len(nodes)) if nodes[i].governor==current_token_idx]
            focused_tokens.update([child_node.token for child_node in child_nodes if child_node.dep == dep_rel])
    return focused_tokens

def extract_targets_using_pattern(token2idx, nodes, opinion_words, dep_rels):

    focused_tokens = opinion_words
    for i in range(len(dep_rels)):
        focused_tokens = next_focused_tokens(focused_tokens, token2idx, nodes, dep_rels[i])
    
    return set(list(focused_tokens))

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt.pr

In [36]:
one_flattened_dep_rels = list(domain_pattern_counter.keys())[0]
dep_rels = one_flattened_dep_rels.split('-')

domain_df['doc'] = domain_df.progress_apply(lambda x: nlp(x['content']), axis=1)

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

In [37]:
def extract_targets(doc, opinion_words, dep_rels):
    targets = set()
    for sentence_from_doc in doc.sentences:
        sentence_graph = DependencyGraph(sentence_from_doc)
        targets.update(extract_targets_using_pattern(sentence_graph.token2idx, sentence_graph.nodes, opinion_words, dep_rels))
    return list(targets)

domain_df['predicted_targets'] = domain_df.parallel_apply(lambda x: extract_targets(x['doc'], x['opinion_words'], dep_rels), axis=1)

/home/dmlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
